# Google and Apple Apps Data Analyis

We look at data from Google and Apple apps, with the goal of answering this question: Which types of apps are most likely to attract the most users on both platforms?

In this notebook we download and clean the data. In the next notebook we will analyze the data.

## I. Data Source

We downloaded the Google apps data from [here](https://www.kaggle.com/lava18/google-play-store-apps/version/6) and the Apple apps data from [here](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps).

In [1]:
# Read data from csv.

from csv import reader

open_file_apple = open('AppleStore.csv', encoding='utf8') 
read_file_apple = reader(open_file_apple)
apple_data = list(read_file_apple)

open_file_google = open('googleplaystore.csv', encoding='utf8')
read_file_google = reader(open_file_google)
google_data = list(read_file_google)

## II. Examine the Data

Let's look at the number of rows and columns in each data set, and a few sample rows to see what the data looks like. 

In [2]:
# Define a function that takes a dataset, and the start and end index for a slice of rows to examine. 
# The output will only display the number of rows and columns if rows_and_columns input is True.
# If the input data has a header, we want to account for that in the row count. 

def explore_data(dataset, start, end, rows_and_columns=False, header=False):  
    
    if rows_and_columns:
        # print number of rows
        if header:
            print('Number of rows with data:', len(dataset)-1) # account for header row by subtracting one from total row count
        else:
            print('Number of rows:', len(dataset))
        
        # print number of columns
        print('Number of columns:', len(dataset[0]))
        print('\n')
    
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

In [3]:
# Explore Google data
# Our data sets both have headers, so we will specify "header = True" in the input for both.

explore_data(google_data, 0, 3, rows_and_columns = True, header = True)

Number of rows with data: 10841
Number of columns: 13


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']




In [4]:
# explore Apple data

explore_data(apple_data, 0, 3, rows_and_columns = True, header = True)

Number of rows with data: 7197
Number of columns: 16


['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']




## III. Data Cleaning

We want to:
- Remove incorrect or incomplete data.
- Remove duplicates.
- Limit data to Enlglish-language apps.
- Limit data to free apps.

### III.a  Remove incomplete data

An entry in the Google dataset with incomplete data is called out in [this thread](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015) in the discussion forum. We will check both data sets for rows that are shorter than the header - an indication that they are missing data - and delete those rows. 

In [5]:
# Check row length discrepancies in google data

n_columns = len(google_data[0])

print(google_data[0])
print('\n')

for row in google_data:
    if len(row) < n_columns:
        print('row index: ' + str(google_data.index(row)))
        print(row)
        print('\n')

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


row index: 10473
['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']




In [6]:
# Check row length discrepancies in apple data

n_columns = len(apple_data[0])

print(apple_data[0])
print('\n')

for row in apple_data:
    if len(row) < n_columns:
        print('row index: ' + str(apple_data.index(row)))
        print(row)
        print('\n')

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']




We identified the same row in the Google data that that discussion forum identified; we delete that in the next cell. We did not find any incomplete rows in the Apple data. 

In [7]:
# Delete row 10473 from google data. 
# NOTE: we only want to delete this once; do not run this row multiple times.

del google_data[10473]

### III.b Remove duplicates

We would like to have only one row per app. To do this we first need to find the duplicate rows, then delete them.

In some cases two rows for the same app are exactly the same, in which case it does not matter which row we keep. In other cases, two rows for the same app may have differing information in other fields. To determine which entry to keep, we decided to keep the row with the highest number of reviews - it is most likely that the entry with the higher number of reviews is the more recent entry.

We will do this in a few steps:
1. **Define a function that identifies duplicates in a given data set.** This function returns both a list of unique app names and a list of the duplicate app names.
2. **Build a dictionary that stores each app name with the highest number of reviews listed for that app in one entry.** We use this as a proxy for the most recent entry for each app. We also write a function for this step. The resulting dictionary has app names as keys and review counts as values.
3. **Create a new data set of unique values by keeping only the entries that match the highest number of reviews for that app, and no identical entries.** We define a function that iterates over our dataset and compares the review count with the app name's corresponding value in the dictionary.

**Define a function that identifies duplicates in a given data set.**

In [8]:
# Define a function to find the duplicates in a given data set.
# Since the app name is in a different column in our two data sets, the function also takes in the index of column that holds the app name.
# The function will return two lists: one of unique app names and one of all duplicate entry names.

def find_duplicate_apps(data, index_app_name):
    unique_apps = []
    duplicate_apps = []
    
    for row in data:
        app_name = row[index_app_name]
        if app_name in unique_apps:
            duplicate_apps.append(app_name)
        else:
            unique_apps.append(app_name)
    
    return unique_apps, duplicate_apps

In [9]:
# Count the number of duplicate rows in the google data.
# Note, we use index 0 as input since that is the column with the app names.

google_duplicates = find_duplicate_apps(google_data, 0)[1]
print(len(google_duplicates))

1181


In [10]:
# Let's look at an example of a duplicated app in the google data.
# From examining the list stored in google_duplicates, I know that the "Slack" app has more than one entry.

for row in google_data:
    if row[0] == 'Slack':
        print(row)
        print('\n')

['Slack', 'BUSINESS', '4.4', '51507', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'August 2, 2018', 'Varies with device', 'Varies with device']


['Slack', 'BUSINESS', '4.4', '51507', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'August 2, 2018', 'Varies with device', 'Varies with device']


['Slack', 'BUSINESS', '4.4', '51510', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'August 2, 2018', 'Varies with device', 'Varies with device']




In [11]:
# Count the number of duplicate rows in the apple data.
# This time we use index 1 for input as that is the column with the app names in this dataset. 

len(find_duplicate_apps(apple_data, 1)[1])

2

**Build a dictionary that stores each app name with the highest number of reviews listed for that app in one entry.**

In [12]:
# Define a function that builds a dictionary to track the greatest review count for each app.
# The keys will be the the app names and the values will be the number of reviews.

def build_max_reviews_dict(data, index_app_name, index_num_reviews):
    max_reviews_dict = {}
    unique_apps = find_duplicate_apps(data, index_app_name)[0] # Use our defined function.
    
    for row in data[1:]:
        app_name = row[index_app_name]
        number_reviews = float(row[index_num_reviews])

        if app_name in unique_apps:
            if (app_name not in max_reviews_dict.keys()) or (number_reviews > max_reviews_dict[app_name]):
                max_reviews_dict[app_name] = number_reviews
    
    return max_reviews_dict

#max_reviews_dict = {}
#google_unique_apps = find_duplicate_apps(google_data, 0)[0] # Use our defined function.

#for row in google_data[1:]:
#    app_name = row[0]
#    number_reviews = float(row[3])
#    
#    if app_name in google_unique_apps:
#        if (app_name not in max_reviews_dict.keys()) or (number_reviews > max_reviews_dict[app_name]):
#            max_reviews_dict[app_name] = number_reviews

In [13]:
google_max_reviews_dict = build_max_reviews_dict(google_data, 0, 3)

print(len(google_data))
print(len(google_max_reviews_dict))

10841
9659


In [14]:
apple_max_reviews_dict = build_max_reviews_dict(apple_data, 1, 5)

print(len(apple_data))
print(len(apple_max_reviews_dict))

7198
7195


**Create a new data set of unique values by keeping only the entries that match the highest number of reviews for that app, and no identical entries.**

In [15]:
# Create new data set without duplicate rows, keeping only the most recent entry for each app 
# (we decided the most recent entryn is the one that has the highest number of reviews)

def remove_duplicates(data, index_app_name, index_num_reviews):
    data_dedup = []
    header = data[0]
    data_dedup.append(header)
    
    already_listed = []
    
    # Use our defined function to build a dictionary of max reviews number.
    max_reviews_dict = build_max_reviews_dict(data, index_app_name, index_num_reviews)
    
    # Iterate over our data and add rows to out new list if they match the max reviews number
    #   AND if they do not already exist in the new list.
    for row in data[1:]:
        app_name = row[index_app_name]
        number_reviews = float(row[index_num_reviews])
        
        if (max_reviews_dict[app_name] == number_reviews) and (app_name not in already_listed):
            data_dedup.append(row)
            already_listed.append(app_name)
            
    return data_dedup

#google_data_new = []
#header = google_data[0]
#google_data_new.append(header)
#
#already_listed = []
#
#for row in google_data[1:]:
#    app_name = row[0]
#    number_reviews = float(row[3])
#    
#    if (max_reviews_dict[app_name] == number_reviews) and (app_name not in already_listed):
#        google_data_new.append(row)
#        already_listed.append(app_name)

In [16]:
google_data_dedup = remove_duplicates(google_data, 0, 3)

len(google_data_dedup)

9660

In [17]:
apple_data_dedup = remove_duplicates(apple_data, 1, 5)

len(apple_data_dedup)

7196

Finally, let's confirm that our functions worked and that there are no duplicates left in either "deduped" data set.

In [18]:
find_duplicate_apps(google_data_dedup, 0)[1]

[]

In [19]:
find_duplicate_apps(google_data_dedup, 0)[1]

[]

### III.c  Limit to English-language apps

We are only interested in including English-language apps for this analysis. One way to approach this is to remove any app that has non-English characters in the app name. 

(Note that this will not necessarily catch apps that are in Spanish, French, or other languages that use predominanely the same alphabet as English, but for the sake of this excercise we will use this approach.)

Our process will take two steps:
1. **Define a function that determines if an app name has more than three non-English characters.** We will iterate over the string to do this, and use the ASCII number of each character to determine if it is in the English alphabet or not. We set the threshold of at least three characters so that emojis and other irregular characters may be permitted if they only occur a few times. 
2. **Apply this function to each row, and remove rows that are not determined to have app names in English**. In order to preserve our data set, rather than delete these rows we will create a new data set for the desired English-only apps. Again, we write a function for this so that it will be easy to apply to both data sets.

In [20]:
def is_string_english(string):
    # keep count of non-English characters in the string
    non_english_char_count = 0
    
    # iterate over the string
    for character in string:
        ascii_num = ord(character)  # use the built-in function ord() to get the ASCII number of a character.
        if ascii_num > 127:  # standard English-language characters will have ASCII number between 0 and 127.
            non_english_char_count += 1
        if non_english_char_count == 3:
            return False  # if the string has more than 3 non-English characters, we determine that the name is not in English.
    return True

In [21]:
def only_english_apps(data, index_app_name):
    data_eng = [data[0]]   
    # eng for "English". I realize in retrospect that this looks like "Data Engineering"...

    for row in data[1:]:
        app_name = row[0]
        if is_string_english(app_name):
            data_eng.append(row)
        
    return data_eng

In [23]:
google_data_eng = only_english_apps(google_data_dedup, 0)

len(google_data_eng)

9598

In [24]:
apple_data_eng = only_english_apps(apple_data_dedup, 0)

len(apple_data_eng)

7196

### III.d  Limit to Free Apps

We are only interested in analyzing apps that are free. In this final dat transformation, we will create a new data set that keeps only the apps for which price = 0.

In [25]:
def only_free_apps(data, index_price):
    data_free = [data[0]]
    
    for row in data[1:]:
        price = float(row[index_price].replace('$',''))
        if price == 0:
            data_free.append(row)
    
    return data_free

In [26]:
google_data_free = only_free_apps(google_data_eng, 7)

print(len(google_data_eng))
print(len(google_data_free))

9598
8849


In [27]:
apple_data_free = only_free_apps(apple_data_eng, 4)

print(len(apple_data))
print(len(apple_data_free))

7198
4055


## IV. Save final data sets to csv

We will use these transformed data sets for analysis in the next notebook.

We follow these instructions [here](https://www.programiz.com/python-programming/writing-csv-files).

In [28]:
from csv import writer

with open('google_data_transformed.csv', 'w', encoding="utf-8", newline='') as file:
    writer_obj = writer(file)
    writer_obj.writerows(google_data_free)

with open('apple_data_transformed.csv', 'w', encoding="utf-8", newline='') as file:
    writer_obj = writer(file)
    writer_obj.writerows(apple_data_free)